In [1]:
import argparse
import time
import torch
from torch.autograd import Variable
import numpy as np
torch.manual_seed(5)

import sys
sys.path.append("..")
# ############################################################################
# param
##############################################################################
batch_size=36
use_cuda=1
##############################################################################
# Load data
##############################################################################
from data_loader import DataLoader
path='/media/kwane/3160053c-937e-4de9-a540-b28bd2802040/kwane/NLP/lstm_text_class/data/'
pretrained_wordvec=np.load(path+'pretrained_wordvec.npy')
data = torch.load(path+'corpus.pt')
max_len = data["max_len"]
vocab_size = data['dict']['vocab_size']
label_size = data['dict']['label_size']

training_data = DataLoader(
             data['train']['src'],
             data['train']['label'],
             data['train']['train_mask'],
             max_len,
             batch_size=batch_size)

validation_data = DataLoader(
              data['valid']['src'],
              data['valid']['label'],
              data['valid']['valid_mask'],
              max_len,
              batch_size=batch_size,
              shuffle=False)

In [2]:
# ##############################################################################
# Build model
# ##############################################################################
import model

global pretrained_wordvec
rnn = model.LSTM_Text(vocab_size=vocab_size,batch_size=batch_size,embed_dim=300,label_size=label_size)
if use_cuda:
    rnn = rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()
print rnn

LSTM_Text(
  (lookup_table): Embedding(9450, 300, padding_idx=0)
  (lstm): LSTM(300, 128, dropout=0.5, bidirectional=1)
  (ln): LayerNorm(
  )
  (logistic): Linear(in_features=256, out_features=6, bias=True)
)


In [4]:
# ##############################################################################
# Training
# ##############################################################################
import time
from tqdm import tqdm

train_loss = []
valid_loss = []
accuracy = []

def repackage_hidden(h):
    if type(h) == Variable:
        if use_cuda:
            return Variable(h.data).cuda()
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

def evaluate():
    rnn.eval()
    corrects = eval_loss = 0
    _size = validation_data.sents_size
    hidden = rnn.init_hidden()
    for data,label,mask in tqdm(validation_data, mininterval=0.2,
                desc='Evaluate Processing', leave=False):
        hidden = repackage_hidden(hidden)
        pred, hidden = rnn(data, hidden)
        loss = criterion(pred, label)

        eval_loss += loss.data
        corrects += (torch.max(pred, 1)[1].view(label.size()).data == label.data).sum()

    return 1.0*eval_loss[0]/_size, corrects, 1.0*corrects/_size * 100.0, _size

def train():
    rnn.train()
    total_loss = 0
    hidden = rnn.init_hidden()
    for data,label,mask in tqdm(training_data, mininterval=1,
                desc='Train Processing', leave=False):
        optimizer.zero_grad()
        hidden = repackage_hidden(hidden)
        target, hidden = rnn(data, hidden)
        loss = criterion(target, label)

        loss.backward()
        optimizer.step()

        total_loss += loss.data
    return 1.0*total_loss[0]/training_data.sents_size



In [13]:
# ##############################################################################
# Save Model
# ##############################################################################
best_acc = None
total_start_time = time.time()

try:
    print('-' * 90)
    for epoch in range(1, 100):
        epoch_start_time = time.time()
        loss = train()
        train_loss.append(loss*1000.)

        print('| start of epoch {:3d} | time: {:2.2f}s | loss {:5.6f}'.format(epoch, time.time() - epoch_start_time, loss))

        loss, corrects, acc, size = evaluate()
        valid_loss.append(loss*1000.)
        accuracy.append(acc)
        print acc
        epoch_start_time = time.time()
        print('-' * 90)
        print('| end of epoch {:3d} | time: {:2.2f}s | loss {:.4f} | accuracy {:.4f}%({}/{})'.format(epoch, time.time() - epoch_start_time, loss, acc, corrects, size))
        print('-' * 90)
        if not best_acc or best_acc < corrects:
            best_acc = corrects
            model_state_dict = rnn.state_dict()
            '''
            model_source = {
                "settings": args,
                "model": model_state_dict,
                "src_dict": data['dict']['train']
            }
            torch.save(model_source, args.save)
            '''
except KeyboardInterrupt:
    print("-"*90)
    print("Exiting from training early | cost time: {:5.2f}min".format((time.time() - total_start_time)/60.0))


Train Processing: 0it [00:00, ?it/s]

------------------------------------------------------------------------------------------
0 36
36 36
72 36
108 36
144 36
180 36
216 36
252 36
288 36
324 36
360 36
396 36
432 36
468 36
504 36
540 36
576 36
612 36
648 36
684 36
720 36
756 36
792 36
828 36
864 36
900 36
936 36
972 36
1008 36
1044 36
1080 36
1116 36
1152 36
1188 36
1224 36
1260 36
1296 36
1332 36
1368 36
1404 36
1440 36
1476 36
1512 36
1548 36
1584 36
1620 36
1656 36
1692 36
1728 36
1764 36
1800 36
1836 36
1872 36
1908 36
1944 36
1980 36
2016 36
2052 36
2088 36
2124 36
2160 36
2196 36
2232 36
2268 36
2304 36
2340 36
2376 36
2412 36
2448 36
2484 36
2520 36
2556 36
2592 36
2628 36
2664 36
2700 36
2736 36
2772 36
2808 36
2844 36
2880 36
2916 36
2952 36
2988 36
3024 36
3060 36
3096 36
3132 36
3168 36
3204 36
3240 36
3276 36
3312 36


Train Processing: 116it [00:01, 115.82it/s]

3348 36
3384 36
3420 36
3456 36
3492 36
3528 36
3564 36
3600 36
3636 36
3672 36
3708 36
3744 36
3780 36
3816 36
3852 36
3888 36
3924 36
3960 36
3996 36
4032 36
4068 36
4104 36
4140 36
4176 36
4212 36
4248 36
4284 36
4320 36
4356 36
4392 36
4428 36
4464 36
4500 36
4536 36
4572 36
4608 36
4644 36
4680 36
4716 36
4752 36
4788 36
4824 36
4860 36
4896 36
4932 36
4968 36
5004 36
5040 36
5076 36


Train Processing: 0it [00:00, ?it/s]       

...

Train Processing: 0it [00:00, ?it/s]   

396 36
432 36
16.4
------------------------------------------------------------------------------------------
| end of epoch  98 | time: 0.00s | loss 0.0448 | accuracy 16.4000%(82/500)
------------------------------------------------------------------------------------------
0 36
36 36
72 36
108 36
144 36
180 36
216 36
252 36
288 36
324 36
360 36
396 36
432 36
468 36
504 36
540 36
576 36
612 36
648 36
684 36
720 36
756 36
792 36
828 36
864 36
900 36
936 36
972 36
1008 36
1044 36
1080 36
1116 36
1152 36
1188 36
1224 36
1260 36
1296 36
1332 36
1368 36
1404 36
1440 36
1476 36
1512 36
1548 36
1584 36
1620 36
1656 36
1692 36
1728 36
1764 36
1800 36
1836 36
1872 36
1908 36
1944 36
1980 36
2016 36
2052 36
2088 36
2124 36
2160 36
2196 36
2232 36
2268 36
2304 36
2340 36
2376 36
2412 36
2448 36
2484 36
2520 36
2556 36
2592 36
2628 36
2664 36
2700 36
2736 36
2772 36
2808 36
2844 36
2880 36
2916 36
2952 36
2988 36
3024 36
3060 36
3096 36
3132 36
3168 36
3204 36
3240 36
3276 36
3312 36
3348 36
3384

Train Processing: 132it [00:01, 131.80it/s]

3852 36
3888 36
3924 36
3960 36
3996 36
4032 36
4068 36
4104 36
4140 36
4176 36
4212 36
4248 36
4284 36
4320 36
4356 36
4392 36
4428 36
4464 36
4500 36
4536 36
4572 36
4608 36
4644 36
4680 36
4716 36
4752 36
4788 36


4824 36
4860 36
4896 36
4932 36
4968 36
5004 36
5040 36
5076 36
5112 36
5148 36
5184 36
5220 36
5256 36
5292 36
5328 36
5364 36
5400 36
| start of epoch  99 | time: 1.15s | loss 0.046083
0 36
36 36
72 36
108 36
144 36
180 36
216 36
252 36
288 36
324 36
360 36
396 36
432 36
16.4
------------------------------------------------------------------------------------------
| end of epoch  99 | time: 0.00s | loss 0.0448 | accuracy 16.4000%(82/500)
------------------------------------------------------------------------------------------


In [6]:
# ##############################################################################
# Training
# ##############################################################################
import time
from tqdm import tqdm

train_loss = []
valid_loss = []
accuracy = []

def repackage_hidden(h):
    if type(h) == Variable:
        if use_cuda:
            return Variable(h.data).cuda()
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

def evaluate():
    rnn.eval()
    corrects = eval_loss = 0
    _size = validation_data.sents_size
    hidden = rnn.init_hidden()
    for data,label,mask in tqdm(validation_data, mininterval=0.2,
                desc='Evaluate Processing', leave=False):
        hidden = repackage_hidden(hidden)
        pred, hidden = rnn(data,mask, hidden)
        loss = criterion(pred, label)

        eval_loss += loss.data
        corrects += (torch.max(pred, 1)[1].view(label.size()).data == label.data).sum()

    return 1.0*eval_loss[0]/_size, corrects, 1.0*corrects/_size * 100.0, _size

def train():
    rnn.train()
    total_loss = 0
    hidden = rnn.init_hidden()
    for data,label,mask in tqdm(training_data, mininterval=1,
                desc='Train Processing', leave=False):
        optimizer.zero_grad()
        hidden = repackage_hidden(hidden)
        target, hidden = rnn(data,mask, hidden)
        loss = criterion(target, label)

        loss.backward()
        optimizer.step()

        total_loss += loss.data
    return 1.0*total_loss[0]/training_data.sents_size



In [28]:
pretrained_wordvec=np.load('./data/pretrained_wordvec.npy')


In [33]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import init
import torch.nn.functional as F

import const


class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super(LayerNorm,self).__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, input):
        mu = torch.mean(input, dim=-1, keepdim=True)
        sigma = torch.std(input, dim=-1, keepdim=True).clamp(min=self.eps)
        output = (input - mu) / sigma
        return output * self.weight.expand_as(output) + self.bias.expand_as(output)

class LSTM_Text(nn.Module):

    def __init__(self,vocab_size,batch_size,embed_dim,label_size):
        super(LSTM_Text,self).__init__()
        global pretrained_wordvec
        self.vocab_size=vocab_size
        self.embed_dim=embed_dim
        self.hidden_size=128
        self.lstm_layers=1
        self.dropout=0.5
        self.batch_size=batch_size
        self.bidirectional=1
        self.label_size=label_size
        #self.num_directions = 2 if self.bidirectional else 1
        self.num_directions = 2 if 1 else 1

        self.lookup_table = nn.Embedding(self.vocab_size, self.embed_dim,
                                padding_idx=const.PAD)
        self.lstm = nn.LSTM(self.embed_dim,
                            self.hidden_size,
                            self.lstm_layers,
                            dropout=self.dropout,
                            bidirectional=self.bidirectional)
        self.ln = LayerNorm(self.hidden_size*self.num_directions)
        self.logistic = nn.Linear(self.hidden_size*self.num_directions,
                                self.label_size)

        self._init_weights()

    def _init_weights(self, scope=1.):
        #self.lookup_table.weight.data.uniform_(-scope, scope)
        self.lookup_table.weight.data.copy_(torch.from_numpy(pretrained_wordvec))
        self.logistic.weight.data.uniform_(-scope, scope)
        self.logistic.bias.data.fill_(0)

    def init_hidden(self):
        num_layers = self.lstm_layers*self.num_directions

        weight = next(self.parameters()).data
        return (Variable(weight.new(num_layers, self.batch_size, self.hidden_size).zero_()),Variable(weight.new(num_layers, self.batch_size, self.hidden_size).zero_()))

    def forward(self, input,mask, hidden):
        
        encode = self.lookup_table(input)
        lstm_out, hidden = self.lstm(encode.transpose(0, 1), hidden)
        #print lstm_out.size(),mask[:, :, None].size(),mask,encode.transpose(0, 1).size()
        #print mask[:, :, None].transpose(0, 1)
        
        output = self.ln(lstm_out)
        final_h = torch.mul(output, mask[:, :, None].transpose(0, 1).type(torch.cuda.FloatTensor))
        final_h=torch.sum(final_h.transpose(0, 1), 1)
        #output = self.ln(lstm_out)[-1]
        #print lstm_out.size(),output.size(),final_h.size()
        return F.log_softmax(self.logistic(final_h)), hidden
# ##############################################################################
# Build model
# ##############################################################################
import model


rnn = LSTM_Text(vocab_size=vocab_size,batch_size=batch_size,embed_dim=300,label_size=label_size)
if use_cuda:
    rnn = rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()
print rnn
# ##############################################################################
# Save Model
# ##############################################################################
best_acc = None
total_start_time = time.time()

try:
    print('-' * 90)
    for epoch in range(1, 100):
        epoch_start_time = time.time()
        loss = train()
        train_loss.append(loss*1000.)

        print('| start of epoch {:3d} | time: {:2.2f}s | loss {:5.6f}'.format(epoch, time.time() - epoch_start_time, loss))

        loss, corrects, acc, size = evaluate()
        valid_loss.append(loss*1000.)
        accuracy.append(acc)
        print acc
        epoch_start_time = time.time()
        print('-' * 90)
        print('| end of epoch {:3d} | time: {:2.2f}s | loss {:.4f} | accuracy {:.4f}%({}/{})'.format(epoch, time.time() - epoch_start_time, loss, acc, corrects, size))
        print('-' * 90)
        if not best_acc or best_acc < corrects:
            best_acc = corrects
            model_state_dict = rnn.state_dict()
            '''
            model_source = {
                "settings": args,
                "model": model_state_dict,
                "src_dict": data['dict']['train']
            }
            torch.save(model_source, args.save)
            '''
except KeyboardInterrupt:
    print("-"*90)
    print("Exiting from training early | cost time: {:5.2f}min".format((time.time() - total_start_time)/60.0))


Train Processing: 0it [00:00, ?it/s]

LSTM_Text(
  (lookup_table): Embedding(9450, 300, padding_idx=0)
  (lstm): LSTM(300, 128, dropout=0.5, bidirectional=1)
  (ln): LayerNorm(
  )
  (logistic): Linear(in_features=256, out_features=6, bias=True)
)
------------------------------------------------------------------------------------------


/media/kwane/3160053c-937e-4de9-a540-b28bd2802040/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:76: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Train Processing: 0it [00:00, ?it/s]       

| start of epoch   1 | time: 1.27s | loss 0.085728
59.6
------------------------------------------------------------------------------------------
| end of epoch   1 | time: 0.00s | loss 0.0278 | accuracy 59.6000%(298/500)
------------------------------------------------------------------------------------------


Train Processing: 0it [00:00, ?it/s]       

...

Train Processing: 0it [00:00, ?it/s]       

| start of epoch  97 | time: 1.17s | loss 0.000272
79.4
------------------------------------------------------------------------------------------
| end of epoch  97 | time: 0.00s | loss 0.0179 | accuracy 79.4000%(397/500)
------------------------------------------------------------------------------------------


Train Processing: 0it [00:00, ?it/s]       

| start of epoch  98 | time: 1.17s | loss 0.000198
80.4
------------------------------------------------------------------------------------------
| end of epoch  98 | time: 0.00s | loss 0.0166 | accuracy 80.4000%(402/500)
------------------------------------------------------------------------------------------


| start of epoch  99 | time: 1.16s | loss 0.000121
80.4
------------------------------------------------------------------------------------------
| end of epoch  99 | time: 0.00s | loss 0.0175 | accuracy 80.4000%(402/500)
------------------------------------------------------------------------------------------
